In [1]:
import pandas as pd
import psycopg2

import os
os.getcwd()

'C:\\Users\\nogur\\Documents\\DeppClinic\\Transferring_Data_to_PostgreSQL'

In [2]:
conn_str = "dbname=postgres user=postgres password=asdf123"


def get_all_table_names(conn_str):
    conn = psycopg2.connect(conn_str)
    sql = """
    SELECT table_name FROM INFORMATION_SCHEMA.TABLES
    WHERE table_schema = N'redcap_data'
    """


    questionnaire_columns = list(pd.read_sql_query(sql, conn).table_name)
    conn.close()
    return questionnaire_columns

## get column names

In [3]:
def get_column_names(questionnaire_name, conn_str):
    conn = psycopg2.connect(conn_str)
    sql = """
    SELECT *
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = N'{0}'
    AND table_schema = N'redcap_data'
    """.format(questionnaire_name)


    questionnaire_columns = list(pd.read_sql_query(sql, conn).column_name)
    conn.close()
    return questionnaire_columns

In [4]:
def prepare_data(questionnaire_name, questionnaire_columns, redcap_df):
    questionnaire_data = redcap_df[questionnaire_columns]

    file_name = "redcap_{0}_data.csv".format(questionnaire_name)
    directory = r"C:/Users/nogur/Documents/DeppClinic/Data/postgres_db/redcap_data"
    full_path = r'{0}/{1}'.format(directory, file_name)
    questionnaire_data.to_csv(full_path, index=False)
    
    # Change the file permissions
    os.chmod(full_path, 0o777)
    
    os.chmod(directory, 0o777)
    

    return full_path


In [5]:
def upload_data(questionnaire_name, full_path, conn_str):
    upload_data_cmd = """
    COPY redcap_data.{0}
    FROM '{1}'
    DELIMITER ','
    CSV HEADER;

    """.format(questionnaire_name, full_path)

    print(upload_data_cmd)
    conn = psycopg2.connect(conn_str)
    # Open a cursor to perform database operations
    cur = conn.cursor()
    # Execute a command: create datacamp_courses table
    cur.execute(upload_data_cmd)
    
    
    # Make the changes to the database persistent
    conn.commit()

    # Close cursor and communication with the database
    cur.close()
    conn.close()

In [6]:
os.chdir("..")
redcap_df = pd.read_csv(r"Data\Transferring_Data_to_PostgreSQL\redcap_data.csv")
table_names = get_all_table_names(conn_str)


C:\Users\nogur\AppData\Local\Temp\ipykernel_14224\2517591147.py:2: DtypeWarning: Columns (44,46,51,53,99,145,325,366,459,464,466,501,507,512,514,516,519,521,530,532,569,571,580,599,766,771,773,779,784,789,794,798,800,804,809,811,813,816,818,827,829,849,866,868,877,896,906,941,964,1002,1014,1025,1063,1068,1070,1076,1081,1086,1091,1093,1095,1097,1099,1100,1126,1142,1149,1165,1168,1172,1173,1177,1178,1179,1180,1182,1183,1184,1187,1188,1190,1207,1212,1229,1232,1260,1262,1264,1266,1268,1270,1271,1281,1283,1285,1287,1289,1291,1293,1295,1297,1299,1301,1303,1305,1306,1319,1321,1324,1326,1336,1338,1340,1342,1343,1344,1350,1351,1352,1353,1459,1465,1467,1468,1470,1486,1489,1492,1496,1513,1515,1516,1517,1523,1524,1525,1526,1580,1650,1653,1654) have mixed types. Specify dtype option on import or set low_memory=False.
  redcap_df = pd.read_csv(r"Data\Transferring_Data_to_PostgreSQL\redcap_data.csv")
C:\Users\nogur\AppData\Local\Temp\ipykernel_14224\3435154038.py:12: UserWarning: pandas only supports

## Prepare the needed table

In [7]:
table_names[-1:]

['intro']

In [8]:

for questionnaire_name in table_names:
    print(questionnaire_name)
    questionnaire_columns = get_column_names(questionnaire_name, conn_str)
    full_path = prepare_data(questionnaire_name, questionnaire_columns, redcap_df)
    upload_data(questionnaire_name, full_path, conn_str)


siq

    COPY redcap_data.siq
    FROM 'C:/Users/nogur/Documents/DeppClinic/Data/postgres_db/redcap_data/redcap_siq_data.csv'
    DELIMITER ','
    CSV HEADER;

    


C:\Users\nogur\AppData\Local\Temp\ipykernel_14224\1858670926.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  questionnaire_columns = list(pd.read_sql_query(sql, conn).column_name)


InsufficientPrivilege: could not open file "C:/Users/nogur/Documents/DeppClinic/Data/postgres_db/redcap_data/redcap_siq_data.csv" for reading: Permission denied
HINT:  COPY FROM instructs the PostgreSQL server process to read a file. You may want a client-side facility such as psql's \copy.


In [ ]:
? os.getcwd